In [1]:
from functools import reduce
from importlib import reload
import numpy as np
import tensornetwork as tn

import converters
import simple_tensors
import twirl
import twirl_optimized
simple_tensors = reload(simple_tensors)
twirl = reload(twirl)
twirl_optimized = reload(twirl_optimized)

# Introduction

This notebook demonstrates the different ways of computing/estimating
$$\int_{\mathbf{U}(2^n)}\!\mathrm{d}U\,\langle i_1 \ldots i_t|U^{\otimes t}|i_1' \ldots i_t'\rangle\langle j_1' \ldots j_t'|U^{\dagger\otimes t}|j_1 \ldots j_t\rangle$$

# Common parameters and data

## Parameters

Number of qubits ($n$ in the equation above).

In [2]:
n_qubits = 2

Number of tensor powers of $U$ in Haar integrations ($t$ in the equation above).

In [3]:
n_tensor_factors = 4

Number of samples (for statistical averaging method).

In [4]:
n_samples = 100

## Computed parameters

In [5]:
d_value = 2 ** n_qubits

## Random bra-kets generation

Generate random indices $i_1, \ldots, i_t, i_1', \ldots, i_t', j_1, \ldots, j_t, j_1', \ldots, j_t'$.

In [6]:
basis_labels_right = np.random.randint(0, d_value, n_tensor_factors)
basis_labels_mid_right = np.random.randint(0, d_value, n_tensor_factors)
basis_labels_mid_left = np.copy(basis_labels_mid_right)
basis_labels_left = np.copy(basis_labels_right)

Comment these to enforce the constraint $i_1 = j_1, \ldots, i_t = j_t$ and $i_1' = j_1', \ldots, i_t' = j_t'$. This makes the terms to be averaged all positive and helps cure the "sign problem". Note that the XEB involves summing averages satisfying this assumption.

In [7]:
"""
np.random.shuffle(basis_labels_mid_left)
np.random.shuffle(basis_labels_left)
"""

'\nnp.random.shuffle(basis_labels_mid_left)\nnp.random.shuffle(basis_labels_left)\n'

# Calculation methods

### Defining $X := |i_1' \ldots i_t'\rangle\langle j_1' \ldots j_t'|$ and computing $\int_{\mathbf{U}(2^n)}\!\mathrm{d}U\,U^{\otimes t}XU^{\dagger \otimes t}$.

In [8]:
X = tn.Node(
    reduce(np.kron, [
        simple_tensors.computational_basis_matrix(d_value, i, j).tensor
        for i, j in zip(basis_labels_mid_left, basis_labels_mid_right)
    ])
    .reshape((d_value, d_value) * n_tensor_factors))

In [9]:
twirl.numerical_haar_average_twirl(X, d_value, n_tensor_factors)[tuple(np.concatenate((basis_labels_left, basis_labels_right)))]

0.0042658730158730155

### Calculating symbolic average of $\langle\phi|U^{\otimes t}|\phi'\rangle\langle\psi'|U^{\dagger\otimes t}|\psi\rangle$ and evaluating for computational basis vectors.

In [10]:
symbolic_average, d_symbol = twirl.symbolic_haar_average_bra_ket(n_tensor_factors)

In [11]:
twirl.numerical_haar_average_computational_basis(
    basis_labels_left,
    basis_labels_mid_left,
    basis_labels_mid_right,
    basis_labels_right,
    d_value,
    symbolic_average,
    d_symbol
)

0.0042658730158730155

### Directly calculating $\int_{\mathbf{U}(2^n)}\!\mathrm{d}U\,\langle i_1 \ldots i_t|U^{\otimes t}|i_1' \ldots i_t'\rangle\langle j_1' \ldots j_t'|U^{\dagger\otimes t}|j_1 \ldots j_t\rangle$ from [1902.08539, theorem 3.1].

In [12]:
twirl_optimized.numerical_haar_average_computational_basis_optimized(
    basis_labels_left,
    basis_labels_mid_left,
    basis_labels_mid_right,
    basis_labels_right,
    d_value
)

0.0042658730158730155

### Statistical averaging

In [13]:
samples = twirl.haar_statistical_sample_computational_basis(
    basis_labels_left,
    basis_labels_mid_left,
    basis_labels_mid_right,
    basis_labels_right,
    d_value,
    n_samples
)
print("mean: {}\nestimated error: {}".format(np.mean(samples), np.std(samples) / np.sqrt(n_samples)))

mean: (0.005372094422333309+0j)
estimated error: 0.0009399179813662027
